Set up the PYTHONPATH so it can find the python modules we'll be using.

In [ ]:
import sys, os
sys.path.append(os.path.join(os.path.pardir, 'python'))

Import the geometry module

In [ ]:
import geometry as geo

Set the base resolution that will be used throughout

In [ ]:
minres = 10.0

In [ ]:
geo.SlabSpline?

In [ ]:
slab = geo.SlabSpline([0.0, 140.0, 240.0, 400.0], [0.0, -70.0, -120.0, -200.0], 
                       res=[2*minres, 1*minres, 1*minres, 6*minres], sid=1, name="Slab")
print(len(slab.points))

In [ ]:
# add partial coupling depths
slab.addpoint(80, "Slab::PartialCouplingDepth", res=1*minres)
slab.addpoint(82.5, "Slab::FullCouplingDepth", res=1*minres)
print(len(slab.points))

In [ ]:
geo.SubductionGeometry?

In [ ]:
geom = geo.SubductionGeometry(slab, slab_rid=1, wedge_rid=2, 
                                    slab_side_sid=2, wedge_side_sid=3, slab_base_sid=7, 
                                    wedge_base_sid=8, coast_sid=11, top_sid=12,
                                    wedge_top_res=4*minres, slab_base_res=6*minres)
geom.plot()

In [ ]:
geom.addcrustlayer(40, "LowerCrust", sid=9, rid=3, slab_res=1*minres, side_res=4*minres, side_sid=5)
print([geom.wedge_side_lines[i].name for i in range(len(geom.wedge_side_lines))])
print(len(slab.points))
geom.plot()

In [ ]:
geom.addcrustlayer(15,"UpperCrust", sid=10, rid=4, slab_res= 2*minres, side_res=4*minres, side_sid=6)
print([geom.wedge_side_lines[i].name for i in range(len(geom.wedge_side_lines))])
geom.plot()

In [ ]:
geom.addwedgesidepoint(139, "WedgeSide::InOut", line_name="UpperWedgeSide", res=4*minres, sid=4)
geom.plot()

In [ ]:
geom.addslabpoint(100, "Slab::DetectorPoint", res=1*minres, sid=1)
geom.plot()

In [ ]:
mesh, ct, ft = geom.generatemesh()

In [ ]:
import dolfinx
import numpy as np

V = dolfinx.fem.FunctionSpace(mesh, ("CG", 2))

In [ ]:
all_slab_dofs = dolfinx.fem.locate_dofs_topological(V, 2, ct.find(1))
all_slab_dofs.size

In [ ]:
all_wedge_dofs = dolfinx.fem.locate_dofs_topological(V, 2, ct.find(2))
all_wedge_dofs.size

In [ ]:
slab_surface_dofs = dolfinx.fem.locate_dofs_topological(V, 1, ft.find(1))
slab_surface_dofs.size

In [ ]:
slab_dofs = np.asarray([dof for dof in all_slab_dofs if dof not in slab_surface_dofs])
slab_dofs.size

In [ ]:
wedge_dofs = np.asarray([dof for dof in all_wedge_dofs if dof not in slab_surface_dofs])
wedge_dofs.size